In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras import regularizers
from hyperas.distributions import uniform


import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [21]:
######### VGG19 parameters ########
dontFreezeLast = 2;

patience = 30;

loadWeights = False;
saveWeights = False;


tensorboard_dir = '../tb/mnist/try1_drop02reg01';
if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)    
checkPointPath = tensorboard_dir + '/best_weights.hdf5';

####################################

gpuName = '/device:GPU:0'
tensorboardFlag = True;
workers = 10;
histogram_freq = 0;

batchSize = 32;
epochs = 50;
validation_size=0.3;

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

print('Training images: ',df_train.shape[0])
print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

('Training images: ', 10222)
('Test images: ', 10357)


In [4]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
im_size = 90

In [8]:
x_train = []
y_train = []

In [9]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 10222/10222 [01:06<00:00, 154.56it/s]


In [10]:
y_train_raw = np.array(y_train, np.uint8)/255.
x_train_raw = np.array(x_train, np.float32) / 255.


We check the shape of the outputs to make sure everyting went as expected.

In [11]:
print(x_train_raw.shape)
print(y_train_raw.shape)

(10222, 90, 90, 3)
(10222, 120)


We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [12]:
num_class = y_train_raw.shape[1]
print('Number of classes: ', num_class)

('Number of classes: ', 120)


It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [13]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [22]:
# Create the base pre-trained model
# Can't download weights in the kernel
with tf.device(gpuName):
    
    dropout_rate = 0.5
    if K.image_data_format() == 'channels_first':
        input_shape = (3, im_size, im_size)
    else:
        input_shape = (im_size, im_size, 3)

     
    
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(im_size,im_size,3)))
    model.add(Activation('relu'))
    BatchNormalization(axis=-1)
    model.add(Conv2D(32, (3, 3),kernel_regularizer=regularizers.l2(0.1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
 
    BatchNormalization(axis=-1)
    model.add(Conv2D(64,(3, 3)))
    model.add(Activation('relu'))
    BatchNormalization(axis=-1)
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    # Fully connected layer

    BatchNormalization()
    model.add(Dense(512))
    model.add(Activation('relu'))
    BatchNormalization()
    model.add(Dropout(0.2))
    model.add(Dense(120))
    #model.add(Conv2D(10,(3,3)))
    #model.add(GlobalAveragePooling2D('none'))
    model.add(Activation('softmax'))
   

    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=patience,
        verbose=1));
    if saveWeights:
        callbacks_list.append(keras.callbacks.ModelCheckpoint(
            checkPointPath,
            monitor='val_acc',
            verbose=1,
            save_best_only=True,
            mode='max',
            save_weights_only=True))
    if tensorboardFlag:
        callbacks_list.append(keras.callbacks.TensorBoard(
                log_dir=tensorboard_dir,
                histogram_freq=histogram_freq,
                write_graph=False,
                write_images=False));
        print('Tensorboard activated in directory: ',tensorboard_dir)
    else:
        print('Tensorboard NOT activated')


    model.summary()

('Tensorboard activated in directory: ', '../tb/mnist/try1_drop02reg01')
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 88, 88, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 88, 88, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 86, 86, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 86, 86, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 41, 41, 64)        18496     
_________________________________________________________________
act

In [23]:
def generator(X,Y,batch_size):
    batch_features = np.ndarray(shape=(batch_size,) + X.shape[1:],
                                dtype=X.dtype);
    batch_labels = np.ndarray(shape=(batch_size,) + Y.shape[1:],
                                dtype=Y.dtype)
    N = X.shape[0];
    
    while True:
        for i in range(batch_size):
            # choose random index in features
            index= np.random.choice(N,1)
            batch_features[i] = X[index]
            batch_labels[i] = Y[index]
        yield batch_features, batch_labels

In [24]:
model.fit(X_train, Y_train,
          epochs=epochs,
          batch_size = batchSize,
          validation_data=(X_valid, Y_valid),
          verbose=1,
          callbacks=callbacks_list)

# steps_per_epoch = round(X_train.shape[0]/batchSize)
# model.fit_generator(generator(X_train,Y_train,batchSize),
#                    steps_per_epoch=steps_per_epoch,
#                    epochs=epochs,
#                    verbose=1,
#                    callbacks=callbacks_list,
#                    validation_data=(X_valid,Y_valid),
#                    workers=workers,
#                    use_multiprocessing=True)

Train on 7155 samples, validate on 3067 samples
Epoch 1/50
7155/7155 [==============================] - 18s 3ms/step - loss: 0.4424 - acc: 0.0078 - val_loss: 0.0188 - val_acc: 0.0137
Epoch 2/50
7155/7155 [==============================] - 18s 3ms/step - loss: 0.0188 - acc: 0.0099 - val_loss: 0.0188 - val_acc: 0.0137
Epoch 3/50
7155/7155 [==============================] - 18s 3ms/step - loss: 0.0187 - acc: 0.0112 - val_loss: 0.0188 - val_acc: 0.0137
Epoch 4/50
7155/7155 [==============================] - 18s 3ms/step - loss: 0.0187 - acc: 0.0116 - val_loss: 0.0188 - val_acc: 0.0137
Epoch 5/50
7155/7155 [==============================] - 18s 3ms/step - loss: 0.0187 - acc: 0.0103 - val_loss: 0.0188 - val_acc: 0.0137
Epoch 6/50
7155/7155 [==============================] - 18s 3ms/step - loss: 0.0187 - acc: 0.0096 - val_loss: 0.0188 - val_acc: 0.0137
Epoch 7/50
7155/7155 [==============================] - 18s 3ms/step - loss: 0.0187 - acc: 0.0098 - val_loss: 0.0188 - val_acc: 0.0137
Epoch 8